# Time Recording Application Testing Code

This notebook is designed to communicate with the Time Recording Application server and perform a series of tests. It is necessary to restart the server for each test to ensure that the test did not fail because of the state the server was in due to the previous tests. Each time you restart the server ensure that it begins with a database in a known state.

In [1]:
# The modules needed
import random
import requests
import shutil
import json
import subprocess

import os
import socket

In [2]:
# The core code

def request2server_get(url,cookies):
    """Send a get request to the server"""
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.get(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]),timeout=30)
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        act=json.loads(r.text)
        return [[ucookie,mcookie],act]
    except :
        print("Invalid login")
        return [[ucookie,mcookie],[]]

def request2server_post(url,cookies,content):
    """Send a post request to the server"""
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.post(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]),json=content, timeout=30)
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        act=json.loads(r.text)
        return [[ucookie,mcookie],act]
    except :
        print("Invalid login")
        return [[ucookie,mcookie],[]]

def do_login(cookies,user,pasw):
    """Send a login command"""
    global server_port
    content = {"command":"login","username":user,"password":pasw}
    return request2server_post("http://localhost:"+server_port+"/action?command=login",cookies, content)

def do_logout(cookies):
    """Send a logout command"""
    global server_port
    content = {"command":"logout"}
    return request2server_post("http://localhost:"+server_port+"/action?command=logout",cookies, content)


def do_get_my_skills(cookies):
    """Send an add_activity command"""
    global server_port
    content = {"command":"get_my_skills"}
    return request2server_post("http://localhost:"+server_port+"/action?command=get_my_skills",cookies,content)

def do_get_upcoming(cookies):
    """Send an add_activity command"""
    global server_port
    content = {"command":"get_upcoming"}
    return request2server_post("http://localhost:"+server_port+"/action?command=get_upcoming",cookies,content)

def do_join_class(cookies,classid):
    """Send an add_activity command"""
    global server_port
    content = {"command":"join_class", "id":classid}
    return request2server_post("http://localhost:"+server_port+"/action?command=join_class",cookies,content)

def do_leave_class(cookies,classid):
    """Send an add_activity command"""
    global server_port
    content = {"command":"leave_class", "id":classid}
    return request2server_post("http://localhost:"+server_port+"/action?command=leave_class",cookies,content)

def do_get_class(cookies,classid):
    """Send an add_activity command"""
    global server_port
    content = {"command":"get_class", "id":classid}
    return request2server_post("http://localhost:"+server_port+"/action?command=get_class",cookies,content)

def do_update_attendee(cookies,attendeeid,state):
    """Send an add_activity command"""
    global server_port
    content = {"command":"update_attendee", "id":attendeeid, "state":state}
    return request2server_post("http://localhost:"+server_port+"/action?command=update_attendee",cookies,content)

def do_cancel_class(cookies,classid):
    """Send an add_activity command"""
    global server_port
    content = {"command":"update_attendee", "id":classid}
    return request2server_post("http://localhost:"+server_port+"/action?command=cancel_class",cookies,content)

def do_create_class(cookies,skillid,note,mx,day,month,year,hour,minute):
    """Send an add_activity command"""
    global server_port
    content = {"command":"create_class", "id": skillid, "note": note, "max":mx, "day":day, "month":month, "year":year, "hour":hour, "minute":minute}
    return request2server_post("http://localhost:"+server_port+"/action?command=create_class",cookies,content)


In [3]:
def do_login_incomplete(cookies,pasw):
    """Send an incomplete login command"""
    global server_port
    content = {"command":"login","password":pasw}
    return request2server_post("http://localhost:"+server_port+"/action?command=login",cookies, content)

def do_join_class_incomplete(cookies):
    """Send an incomplete add_activity command"""
    global server_port
    content = {"command":"join_class"}
    return request2server_post("http://localhost:"+server_port+"/action?command=join_class",cookies,content)

def do_leave_class_incomplete(cookies):
    """Send an incomplete add_activity command"""
    global server_port
    content = {"command":"leave_class"}
    return request2server_post("http://localhost:"+server_port+"/action?command=leave_class",cookies,content)

def do_get_class_incomplete(cookies):
    """Send an incomplete add_activity command"""
    global server_port
    content = {"command":"get_class"}
    return request2server_post("http://localhost:"+server_port+"/action?command=get_class",cookies,content)

def do_cancel_class_incomplete(cookies):
    """Send an incomplete add_activity command"""
    global server_port
    content = {"command":"cancel_class"}
    return request2server_post("http://localhost:"+server_port+"/action?command=cancel_class",cookies,content)

def do_create_class_incomplete(cookies,skillid,mx,day,month,year,hour,minute):
    """Send an add_activity command"""
    global server_port
    content = {"command":"create_class", "id": skillid, "max":mx, "day":day, "month":month, "year":year, "hour":hour, "minute":minute}
    return request2server_post("http://localhost:"+server_port+"/action?command=create_class",cookies,content)


In [4]:
def find_redirect(act):
    """Check for a redirect response. Return the where target if found or None otherwise."""
    if act == None:
        return None
    try:
        for a in act:
            if(a['type']=='redirect'):
                return a['where']
    except:
        return None

def find_message_zero(act):
    """Check if there is a message with code 0"""
    if act == None:
        return False
    try:
        for a in act:
            if(a['type']=='message'):
                code = a['code']
                if code == 0:
                    return True
    except:
        return False
    
def find_message_100(act):
    """Check if there is a message with code in the range 100-199"""
    if act == None:
        return False
    try:
        for a in act:
            if(a['type']=='message'):
                code = a['code']
                if code >= 100 and code <= 199:
                    return True
    except:
        return False
    
def find_message_200(act):
    """Check if there is a message with code in the range 200-299"""
    if act == None:
        return False
    try:
        for a in act:
            if(a['type']=='message'):
                code = a['code']
                if code >= 200 and code <= 299:
                    return True
    except:
        return False

def checked_login(test,cookies,user,pasw):
    """Send a login command and check it's good."""
    [cookies,act] = do_login(cookies, user,pasw)
    where = find_redirect(act)
    if(where == None):
        print("Test "+str(test)+" Failed - Expected redirect during login.")
        return ['',act,False]
    if (where != '/index.html'):
        print("Test "+str(test)+" Failed - Expected /index.html got {"+where+"}")
        return [cookies,act,False]
    return [cookies,act,True]

def checked_logout(test,cookies):
    """Send a logout command and check it's good."""
    [cookies,act] = do_logout(cookies)     
    where = find_redirect(act)
    if(where == None):
        print("Test "+str(test)+" Failed - Expected redirect during logout.")
        return [cookies,act,False]
    if (where != '/logout.html'):
        print("Test "+str(test)+" Failed - Expected /logout.html got {"+where+"}")
        return [cookies,act,False] 
    return [cookies,act,True]

In [5]:
# Test 1 - Simple login
def test1():
    """Check that login and logout work for a good user."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(1,cookies,"test1","pass1word")
        if flag!=True:
            return 0

        [cookies,act,flag] = checked_logout(1,cookies)        
        if flag!=True:
            return 0

        print("Test 1 Passed")
        return 1
    except:
        print("Test 1 Failed - Exception Caused.")
        return 0


In [6]:
# Test 2 - Login with bad password
def test2():
    """Check that login reports bad parameters, in this case a bad password."""
    try:
        cookies = ['','']
        [cookies,act] = do_login(cookies,"test1","notpassword")
        
        where = find_redirect(act)
        if (where != None):
            print("Test 2 Failed - Unexpected redirect.")
            return 0
        
        if find_message_100(act):
            print("Test 2 Failed - Unexpected message code found.")
            return 0
        
        if find_message_200(act) == False:
            print("Test 2 Failed - Bad parameter message not found.")
            return 0

        print("Test 2 Passed")
        return 1
    except:
        print("Test 2 Failed - Exception Caused.")
        return 0

In [7]:
# Test 3 - Login with missing username
def test3():
    """Check that login rejects attempts with missing parameters, in this case a missing username"""
    try:
        cookies = ['','']
        [cookies,act] = do_login_incomplete(cookies,"passwordX")
        
        where = find_redirect(act)
        if (where != None):
            print("Test 3 Failed - Unexpected redirect.")
            return 0
        
        if find_message_200(act):
            print("Test 3 Failed - Unexpected bad parameter message code found.")
            return 0
        
        if find_message_zero(act):
            print("Test 3 Failed - Unexpected all good message found.")
            return 0
        
        if find_message_100(act) == False:
            print("Test 3 Failed - Missing parameter message not found.")
            return 0
        print("Test 3 Passed")
        return 1
    except:
        print("Test 3 Failed - Exception Caused.")
        return 0        

In [8]:
# Test 4 - check two session magic are different
def test4():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(4,cookies,"test1","pass1word")
        
        cookies2 = ['','']
        [cookies2,act,flag] = checked_login(44,cookies2,"test2","pass2word")

        if cookies[1] == cookies2[1]:
            print("Test 4 failed - Repeated cookie magic detected.")
        
        
        [cookies,act,flag] = checked_logout(4,cookies)
        [cookies,act,flag] = checked_logout(4,cookies2)
        
        print("Test 4 Passed")
    
        return 1
    except:
        print("Test 4 Failed - Exception Caused.")
        return 0

In [9]:
# Test 5 - Get upcoming, when classes present
def test5():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(5,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_upcoming(cookies)
      
        if len(act)==0:
            print("Test 5 Failed - Expected some responses")
            return 0
        
        acount = 0
        mask = 0
        
        try:
            for a in act:
                if a['type'] == 'class':
                    acount += 1
                    if a['name']=='Laser Cutter' and a['id']==3:
                        mask = mask | 1
                    if a['name']=='Wood Lathe' and a['id']==5:
                        mask = mask | 2
                    if a['name']=='Laser Cutter' and a['id']==2:
                        mask = mask | 4
                elif a['type'] == 'message':
                    if a['code'] != 0:
                        print("Test 5 Failed - Message other than success found.")
                        return 0
            if find_message_zero(act) == False:
                print("Test 5 Failed - No success message found.")
                return 0
                        
        except:
            print("Test 5 Failed - Responses missing elements.")
            return 0
        
        if acount != 3 :
            print("Test 5 Failed - Expected number of class responses not found.")
            return 0
        
        if mask != 7:
            print ("Test 5 Failed - Not the three classes that were expected.")
            return 0
        
        print("Test 5 Passed")
        return 1
    except:
        print("Test 5 Failed - Exception Caused.")
        return 0

In [10]:
# Test 6 - Get upcoming, when no classes present
def test6():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(6,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_upcoming(cookies)
      
        if len(act)==0:
            print("Test 6 Failed - Expected some responses")
            return 0
        
        acount = 0
        
        try:
            for a in act:
                if a['type'] == 'class':
                    acount += 1
                elif a['type'] == 'message':
                    if a['code'] != 0:
                        print("Test 6 Failed - Message other than success found.")
                        return 0
                    
            if find_message_zero(act) == False:
                print("Test 6 Failed - No success message found.")
                return 0
                        
        except:
            print("Test 6 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 6 Failed - Expected number of class responses not found.")
            return 0

        
        print("Test 6 Passed")
        return 1
    except:
        print("Test 6 Failed - Exception Caused.")
        return 0

In [11]:
# Test 7 - Get upcoming, when no future uncancelled classes present
def test7():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(7,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_upcoming(cookies)
      
        if len(act)==0:
            print("Test 7 Failed - Expected some responses")
            return 0
        
        acount = 0
        
        try:
            for a in act:
                if a['type'] == 'class':
                    acount += 1
                elif a['type'] == 'message':
                    if a['code'] != 0:
                        print("Test 7 Failed - Expected success message not present.")
                        return 0
                    
            if find_message_zero(act) == False:
                print("Test 7 Failed - No success message found.")
                return 0
                        
        except:
            print("Test 7 Failed - Responses missing elements.")
            return 0
        
        if acount != 1 :
            print("Test 7 Failed - Expected number of class responses not found.")
            return 0
        

        
        print("Test 7 Passed")
        return 1
    except:
        print("Test 7 Failed - Exception Caused.")
        return 0

In [12]:
# Test 8 - Join a class
def test8():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(8,cookies,"user8","pass8word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_join_class(cookies, 2)
      
        if len(act)==0:
            print("Test 8 Failed - Expected some responses")
            return 0
        
        acount = 0
        
        try:
            for a in act:
                if a['type'] == 'class':
                    acount += 1
                    if a['id'] != 2 or a['size'] != 2 or a['max'] != 10 or a['when'] != 1917111720:
                        print("Test 8 Failed - No class details expected")
                        return 0
                elif a['type'] == 'message':
                    if a['code'] != 0:
                        print("Test 8 Failed - Non success message found.")
                        return 0
                    
            if find_message_zero(act) == False:
                print("Test 8 Failed - No success message found.")
                return 0
                        
        except:
            print("Test 8 Failed - Responses missing elements.")
            return 0
        
        if acount != 1 :
            print("Test 8 Failed - Expected number of class responses not found.")
            return 0
        
        print("Test 8 Passed")
        return 1
    except:
        print("Test 8 Failed - Exception Caused.")
        return 0

In [13]:
# Test 9 - Join a class non existant class
def test9():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(9,cookies,"user8","pass8word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_join_class(cookies, 20)
      
        if len(act)==0:
            print("Test 9 Failed - Expected some responses")
            return 0
        
        acount = 0
        
        try:
            for a in act:
                if a['type'] == 'class':
                    acount += 1
                elif a['type'] == 'message':
                    if a['code'] == 0:
                        print("Test 9 Failed - Expected failure message not present.")
                        return 0
                    
            if find_message_200(act) == False:
                print("Test 9 Failed - Bad parameter message not found.")
                return 0
        
            if find_message_zero(act):
                print("Test 9 Failed - Success message found.")
                return 0
        
            if find_message_100(act):
                print("Test 2 Failed - Missing parameter found.")
                return 0
                        
        except:
            print("Test 9 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 9 Failed - Expected number of class responses not found.")
            return 0

        
        print("Test 9 Passed")
        return 1
    except:
        print("Test 9 Failed - Exception Caused.")
        return 0

In [14]:
# Test 10 - Join a class that's already full
def test10():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(10,cookies,"user8","pass8word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_join_class(cookies, 5)
      
        if len(act)==0:
            print("Test 10 Failed - Expected some responses")
            return 0
        
        acount = 0
        
        try:
            for a in act:
                if a['type'] == 'class':
                    acount += 1
                    
            if find_message_200(act) == False:
                print("Test 10 Failed - Bad parameter message not found.")
                return 0
        
            if find_message_zero(act):
                print("Test 10 Failed - Success message found.")
                return 0
        
            if find_message_100(act):
                print("Test 10 Failed - Missing parameter found.")
                return 0
                        
        except:
            print("Test 10 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 910Failed - Expected number of class responses not found.")
            return 0

        
        print("Test 10 Passed")
        return 1
    except:
        print("Test 10 Failed - Exception Caused.")
        return 0

In [15]:
# Test 11 - Join a class with missing parameter
def test11():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(11,cookies,"user8","pass8word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_join_class_incomplete(cookies)
      
        if len(act)==0:
            print("Test 11 Failed - Expected some responses")
            return 0
        
        acount = 0
        
        try:
            for a in act:
                if a['type'] == 'class':
                    acount += 1
                    
            if find_message_100(act) == False:
                print("Test 11 Failed - Missing parameter message not found")
                return 0
        
            if find_message_zero(act):
                print("Test 11 Failed - Success message found.")
                return 0
        
            if find_message_200(act):
                print("Test 11 Failed - Bad parameter message found")
                return 0
                        
        except:
            print("Test 11 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 11 Failed - Expected number of class responses not found.")
            return 0
        
        print("Test 11 Passed")
        return 1
    except:
        print("Test 11 Failed - Exception Caused.")
        return 0

In [16]:
# Test 12 - Leave a class with missing parameter
def test12():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(12,cookies,"user8","pass8word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_leave_class_incomplete(cookies)
      
        if len(act)==0:
            print("Test 12 Failed - Expected some responses")
            return 0
        
        acount = 0
        
        try:
            for a in act:
                if a['type'] == 'class':
                    acount += 1

            if find_message_100(act) == False:
                print("Test 12 Failed - Missing parameter message not found")
                return 0
        
            if find_message_zero(act):
                print("Test 12 Failed - Success message found.")
                return 0
        
            if find_message_200(act):
                print("Test 12 Failed - Bad parameter message found")
                return 0                
        except:
            print("Test 12 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 12 Failed - Expected number of class responses not found.")
            return 0

        print("Test 12 Passed")
        return 1
    except:
        print("Test 12 Failed - Exception Caused.")
        return 0

In [17]:
# Test 13 - Everyone's a winner
def test13():
    print("Test 13 Passed")
    return 1

In [18]:
# Test 14 - Get class detail, simple case
def test14():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(14,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_class(cookies,1)
      
        if len(act)==0:
            print("Test 14 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        mask = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                    if a['id'] == 1 and a['action']=="failed":
                        mask = mask | 1
                    if a['id'] == 3 and a['action']=="passed":
                        mask = mask | 2
                if a['type'] == 'class':
                    ccount += 1
                elif a['type'] == 'message':
                    if a['code'] != 0:
                        print("Test 14 Failed - Unexpected failure message present")
                        return 0

        except:
            print("Test 14 Failed - Responses missing elements.")
            return 0
        
        if acount != 2 :
            print("Test 14 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 1 :
            print("Test 14 Failed - Expected number of class responses not found.")
            return 0
        
        if mask != 3:
            print("Test 14 Failed - Attendee status not as expected.")
            return 0
        
        if find_message_zero(act) == False:
            print("Test 14 Failed - Success message not found.")
            return 0
        
        print("Test 14 Passed")
        return 1
    except:
        print("Test 14 Failed - Exception Caused.")
        return 0

In [19]:
# Test 15 - Get class detail, with no attendees
def test15():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(15,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_class(cookies,3)
      
        if len(act)==0:
            print("Test 15 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                elif a['type'] == 'message':
                    if a['code'] != 0:
                        print("Test 15 Failed - Non-success message found.")
                        return 0

            if find_message_zero(act) == False:
                print("Test 15 Failed - Success message not found.")
                return 0
                        
        except:
            print("Test 15 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 15 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 1 :
            print("Test 15 Failed - Expected number of class responses not found.")
            return 0

        
        print("Test 15 Passed")
        return 1
    except:
        print("Test 15 Failed - Exception Caused.")
        return 0

In [20]:
# Test 16 - Get class detail, not belonging to user
def test16():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(16,cookies,"test2","pass2word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_class(cookies,1)
      
        if len(act)==0:
            print("Test 16 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 16 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 16 Failed - Success message found.")
                return 0
        
            if find_message_200(act)== False:
                print("Test 16 Failed - Bad parameter message not found")
                return 0 
                        
        except:
            print("Test 16 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 16 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 16 Failed - Expected number of class responses not found.")
            return 0

        
        print("Test 16 Passed")
        return 1
    except:
        print("Test 16 Failed - Exception Caused.")
        return 0

In [21]:
# Test 17 - Get class detail, not an existing class
def test17():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(17,cookies,"test2","pass2word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_class(cookies,100)
      
        if len(act)==0:
            print("Test 17 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 17 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 17 Failed - Success message found.")
                return 0
        
            if find_message_200(act)== False:
                print("Test 17 Failed - Bad parameter message not found")
                return 0 
                        
        except:
            print("Test 17 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 17 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 17 Failed - Expected number of class responses not found.")
            return 0
        
        print("Test 17 Passed")
        return 1
    except:
        print("Test 17 Failed - Exception Caused.")
        return 0

In [22]:
# Test 18 - Set update ok
def test18():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(18,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_update_attendee(cookies,1,"pass")
      
        if len(act)==0:
            print("Test 18 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                    if a['id'] != 1 or a['action']!="passed":
                        print("Test 18 Failed: attendee id or action not as expected")
                        return 0
                if a['type'] == 'class':
                    ccount += 1
                elif a['type'] == 'message':
                    if a['code'] != 0:
                        print("Test 18 Failed - A message other than success found.")
                        return 0
                    
            if find_message_zero(act) == False:
                print("Test 19 Failed - Success message not found.")
                return 0
                        
        except:
            print("Test 18 Failed - Responses missing elements.")
            return 0
        
        if acount != 1 :
            print("Test 18 Failed - Expected number of attendee responses not found.")
            return 0
        # not checking for class entry as spec is not clear
        
        print("Test 18 Passed")
        return 1
    except:
        print("Test 18 Failed - Exception Caused.")
        return 0

In [23]:
# Test 19 - Set update when not trainer.
def test19():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(19,cookies,"user8","pass8word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_update_attendee(cookies,1,"pass")
      
        if len(act)==0:
            print("Test 19 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 19 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 19 Failed - Success message found.")
                return 0
        
            if find_message_200(act)== False:
                print("Test 19 Failed - Bad parameter message not found")
                return 0 
                        
        except:
            print("Test 19 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 19 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 19 Failed - Expected number of class responses not found.")
            return 0

        
        print("Test 19 Passed")
        return 1
    except:
        print("Test 19 Failed - Exception Caused.")
        return 0

In [24]:
# Test 20 - Set update to soon
def test20():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(20,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_update_attendee(cookies,2,"pass")
      
        if len(act)==0:
            print("Test 20 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 20 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 20 Failed - Success message found.")
                return 0
        
            if find_message_200(act)== False:
                print("Test 20 Failed - Bad parameter message not found")
                return 0 

                        
        except:
            print("Test 20 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 20 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 20 Failed - Expected number of class responses not found.")
            return 0

        
        print("Test 20 Passed")
        return 1
    except:
        print("Test 20 Failed - Exception Caused.")
        return 0

In [25]:
# Test 21 - Cancel future class as trainer
def test21():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(21,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_cancel_class(cookies,3)
      
        if len(act)==0:
            print("Test 21 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                    if a['id'] != 3 or a['size'] != 0 or a['max'] != 0:
                        print("Test 21 failed - Details of cancelled class not as expected.")
                elif a['type'] == 'message':
                    if a['code'] != 0:
                        print("Test 21 Failed - Expected success message not present.")
                        return 0
                    
            if find_message_zero(act) == False:
                print("Test 21 Failed - Success message not found.")
                return 0
                        
        except:
            print("Test 21 Failed - Responses missing elements.")
            return 0
        
        # not checking attendee count
        
        if ccount != 1 :
            print("Test 21 Failed - Expected number of class responses not found.")
            return 0
        
        print("Test 21 Passed")
        return 1
    except:
        print("Test 21 Failed - Exception Caused.")
        return 0

In [26]:
# Test 22 - Cancel future class as wrong user
def test22():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(22,cookies,"test5","pass5word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_cancel_class(cookies,3)
      
        if len(act)==0:
            print("Test 21 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 22 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 22 Failed - Success message found.")
                return 0
        
            if find_message_200(act)== False:
                print("Test 22 Failed - Bad parameter message not found")
                return 0 

                        
        except:
            print("Test 22 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 22 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 22 Failed - Expected number of class responses not found.")
            return 0

        
        print("Test 22 Passed")
        return 1
    except:
        print("Test 21 Failed - Exception Caused.")
        return 0

In [27]:
# Test 23 - Cancel future class too late
def test23():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(23,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_cancel_class(cookies,1)
      
        if len(act)==0:
            print("Test 23 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 23 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 23 Failed - Success message found.")
                return 0
        
            if find_message_200(act)== False:
                print("Test 23 Failed - Bad parameter message not found")
                return 0 

                        
        except:
            print("Test 23 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 23 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 23 Failed - Expected number of class responses not found.")
            return 0
        
        print("Test 23 Passed")
        return 1
    except:
        print("Test 22 Failed - Exception Caused.")
        return 0

In [28]:
# Test 24 - Cancel future class too late
def test24():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(24,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_cancel_class_incomplete(cookies)
      
        if len(act)==0:
            print("Test 24 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act) == False:
                print("Test 24 Failed - Missing parameter message not found")
                return 0
        
            if find_message_zero(act):
                print("Test 24 Failed - Success message found.")
                return 0
        
            if find_message_200(act):
                print("Test 24 Failed - Bad parameter message found")
                return 0 
                        
        except:
            print("Test 24 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 24 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 24 Failed - Expected number of class responses not found.")
            return 0
        
        print("Test 24 Passed")
        return 1
    except:
        print("Test 24 Failed - Exception Caused.")
        return 0

In [29]:
# Test 25 - Cancel invalid class as trainer
def test25():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(25,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_cancel_class(cookies,30)
      
        if len(act)==0:
            print("Test 25 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 20 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 20 Failed - Success message found.")
                return 0
        
            if find_message_200(act) == False:
                print("Test 20 Failed - Bad parameter message not found")
                return 0 
                        
        except:
            print("Test 25 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 25 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 25 Failed - Expected number of class responses not found.")
            return 0
        
        print("Test 25 Passed")
        return 1
    except:
        print("Test 25 Failed - Exception Caused.")
        return 0

In [30]:
# Test 26 - Create a class successfully
def test26():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(26,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_create_class(cookies,1,"hello",5,1,2,2025,10,30)
      
        if len(act)==0:
            print("Test 26 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        mcount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'redirect':
                    rcount += 1
                    if a['where'] != '/class/9':
                        printf("Test 26 Failed - Redirect to correct class not found")
                        return 0
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 26 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 26 Failed - Success message not expected when redirecting.")
                return 0
        
            if find_message_200(act):
                print("Test 26 Failed - Bad parameter message found")
                return 0 

                        
        except:
            print("Test 26 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 26 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 26 Failed - Expected number of class responses not found.")
            return 0
        if rcount != 1 :
            print("Test 26 Failed - Expected number of redirect response not found.")
            return 0
        
        print("Test 26 Passed")
        return 1
    except:
        print("Test 26 Failed - Exception Caused.")
        return 0

In [31]:
# Test 27 - Create a class, fail not trainer
def test27():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(27,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_create_class(cookies,2,"hello",5,1,2,2025,10,30)
      
        if len(act)==0:
            print("Test 27 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'redirect':
                    rcount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 20 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 20 Failed - Success message found.")
                return 0
        
            if find_message_200(act)== False:
                print("Test 20 Failed - Bad parameter message not found")
                return 0 
                        
        except:
            print("Test 27 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 27 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 27 Failed - Expected number of class responses not found.")
            return 0
        if rcount != 0 :
            print("Test 27 Failed - Expected number of redirect responses not found.")
            return 0
        
        print("Test 27 Passed")
        return 1
    except:
        print("Test 27 Failed - Exception Caused.")
        return 0

In [32]:
# Test 28 - Create a class invalid day
def test28():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(28,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_create_class(cookies,1,"hello",5,50,2,2025,10,30)
      
        if len(act)==0:
            print("Test 28 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'redirect':
                    rcount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 28 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 28 Failed - Success message found.")
                return 0
        
            if find_message_200(act)== False:
                print("Test 28 Failed - Bad parameter message not found")
                return 0 

        except:
            print("Test 28 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 28 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 28 Failed - Expected number of class responses not found.")
            return 0
        if rcount != 0 :
            print("Test 28 Failed - Expected number of redirect responses not found.")
            return 0
        
        print("Test 28 Passed")
        return 1
    except:
        print("Test 28 Failed - Exception Caused.")
        return 0

In [33]:
# Test 29 - Create a class, missing note
def test29():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(29,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_create_class_incomplete(cookies,1,5,5,2,2025,10,30)
      
        if len(act)==0:
            print("Test 29 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'redirect':
                    rcount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act) == False:
                print("Test 29 Failed - Missing parameter message not found")
                return 0
        
            if find_message_zero(act):
                print("Test 29 Failed - Success message found.")
                return 0
        
            if find_message_200(act):
                print("Test 29 Failed - Bad parameter message found")
                return 0 

                        
        except:
            print("Test 29 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 29 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 29 Failed - Expected number of class responses not found.")
            return 0
        if rcount != 0 :
            print("Test 29 Failed - Expected number of redirect responses not found.")
            return 0
        
        print("Test 29 Passed")
        return 1
    except:
        print("Test 29 Failed - Exception Caused.")
        return 0

In [34]:
# Test 30 - Create a class, not a valid skill
def test30():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(30,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_create_class(cookies,10,"hello",5,5,2,2025,10,30)
      
        if len(act)==0:
            print("Test 30 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'redirect':
                    rcount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 30 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act):
                print("Test 30 Failed - Success message found.")
                return 0
        
            if find_message_200(act) == False:
                print("Test 30 Failed - Bad parameter message not found")
                return 0 
                        
        except:
            print("Test 30 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 30 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 30 Failed - Expected number of class responses not found.")
            return 0
        if rcount != 0 :
            print("Test 30 Failed - Expected number of redirect responses not found.")
            return 0
        
        print("Test 30 Passed")
        return 1
    except:
        print("Test 30 Failed - Exception Caused.")
        return 0

In [35]:
# Test 31 - Get skills
def test31():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(31,cookies,"test1","pass1word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_my_skills(cookies)
      
        if len(act)==0:
            print("Test 31 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        scount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'skill':
                    scount += 1
                    if a['gained'] != 1514836800:
                        print("Test 31 Failed - Skill gained date not correct.")
                        return 0
                if a['type'] == 'redirect':
                    rcount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 31 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act)== False:
                print("Test 31 Failed - Success message not found.")
                return 0
        
            if find_message_200(act):
                print("Test 31 Failed - Bad parameter message found")
                return 0 
                        
        except:
            print("Test 31 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 31 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 31 Failed - Expected number of class responses not found.")
            return 0
        if scount != 1 :
            print("Test 31 Failed - Expected number of skill responses not found.")
            return 0
        
        print("Test 31 Passed")
        return 1
    except:
        print("Test 31 Failed - Exception Caused.")
        return 0

In [36]:
# Test 32 - Get skills
def test32():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(32,cookies,"user6","pass6word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_my_skills(cookies)
      
        if len(act)==0:
            print("Test 32 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        scount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'skill':
                    scount += 1
                if a['type'] == 'redirect':
                    rcount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 32 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act) == False:
                print("Test 32 Failed - Success message not found.")
                return 0
        
            if find_message_200(act):
                print("Test 32 Failed - Bad parameter message found")
                return 0 
                        
        except:
            print("Test 32 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 32 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 32 Failed - Expected number of class responses not found.")
            return 0
        if scount != 1 :
            print("Test 32 Failed - Expected number of skill responses not found.")
            return 0
        
        print("Test 32 Passed")
        return 1
    except:
        print("Test 32 Failed - Exception Caused.")
        return 0

In [37]:
# Test 33 - Get skills
def test33():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(33,cookies,"user7","pass7word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_my_skills(cookies)
      
        if len(act)==0:
            print("Test 33 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        scount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'skill':
                    scount += 1
                if a['type'] == 'redirect':
                    rcount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 33 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act) == False:
                print("Test 33 Failed - Success message not found.")
                return 0
        
            if find_message_200(act):
                print("Test 33 Failed - Bad parameter message found")
                return 0 
                        
        except:
            print("Test 33 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 33 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 33 Failed - Expected number of class responses not found.")
            return 0
        if scount != 3 :
            print("Test 33 Failed - Expected number of skill responses not found.")
            return 0
        
        print("Test 33 Passed")
        return 1
    except:
        print("Test 33 Failed - Exception Caused.")
        return 0

In [38]:
# Test 34 - Get skills
def test34():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(34,cookies,"user9","pass9word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_my_skills(cookies)
      
        if len(act)==0:
            print("Test 34 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        scount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'skill':
                    scount += 1
                if a['type'] == 'redirect':
                    rcount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 34 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act) == False:
                print("Test 34 Failed - Success message not found.")
                return 0
        
            if find_message_200(act):
                print("Test 34 Failed - Bad parameter message found")
                return 0 
                        
        except:
            print("Test 34 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 34 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 34 Failed - Expected number of class responses not found.")
            return 0
        if scount != 1 :
            print("Test 34 Failed - Expected number of skill responses not found.")
            return 0
        
        print("Test 34 Passed")
        return 1
    except:
        print("Test 34 Failed - Exception Caused.")
        return 0

In [39]:
# Test 35 - Get skills
def test35():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(35,cookies,"user10","pass10word")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_my_skills(cookies)
      
        if len(act)==0:
            print("Test 35 Failed - Expected some responses")
            return 0
   
        acount = 0
        ccount = 0
        rcount = 0
        scount = 0
        
        try:
            for a in act:
                if a['type'] == 'attendee':
                    acount += 1
                if a['type'] == 'skill':
                    scount += 1
                if a['type'] == 'redirect':
                    rcount += 1
                if a['type'] == 'class':
                    ccount += 1
                    
            if find_message_100(act):
                print("Test 35 Failed - Missing parameter message found")
                return 0
        
            if find_message_zero(act) == False:
                print("Test 35 Failed - Success message not found.")
                return 0
        
            if find_message_200(act):
                print("Test 35 Failed - Bad parameter message found")
                return 0 
        except:
            print("Test 35 Failed - Responses missing elements.")
            return 0
        
        if acount != 0 :
            print("Test 35 Failed - Expected number of attendee responses not found.")
            return 0
        if ccount != 0 :
            print("Test 35 Failed - Expected number of class responses not found.")
            return 0
        if scount != 1 :
            print("Test 35 Failed - Expected number of skill responses not found.")
            return 0
        
        print("Test 35 Passed")
        return 1
    except:
        print("Test 30 Failed - Exception Caused.")
        return 0

In [40]:
def check_pylint(sfile):
    """Compute a score for code quality using pylint."""
    try:
        sp = subprocess.Popen('c:\ProgramData\Anaconda3\Scripts\pylint.exe '+sfile,stdout=subprocess.PIPE)
        report = sp.communicate()[0]
        sp.wait()
        sp.terminate()

        last = ''
        for r in report.splitlines():
            s = r.decode('utf-8')
            if(len(s)>0):
                last = s
        x = last[28:].split('/')
        if(x[1].startswith('10')):
            ps = float(x[0])
            if ps < 0.0:
                ps = 0.0;
        else:
            ps = -1.0;
            
        ps = int(ps)
        if (ps > 4):
            ps = ps - 4
        else:
            ps = 0;        
        return ps
    except:
        return -1.0;



In [41]:
# The lists of tests, you can add your own here.
# test description, test function, database to use for test
tests = [('Test 1 Simple login test.', test1, 'db/just_users.db'),
         ('Test 2 Check bad password rejected.', test2, 'db/just_users.db'),
         ('Test 3 Check missing username on login is rejected', test3, 'db/just_users.db'),
         ('Test 4 Check magic is different', test4, 'db/just_users.db'),
         ('Test 5 Check upcoming when future classes', test5, 'db/with_classes.db'),
         ('Test 6 Check when no classes', test6, 'db/with_just_skills.db'),
         ('Test 7 Check when no future classes', test7, 'db/with_no_future_classes.db'),
         ('Test 8 Check join class', test8, 'db/with_classes.db'),
         ('Test 9 Check join non-existant class', test9, 'db/with_classes.db'),
         ('Test 10 Check join full class', test10, 'db/with_classes.db'),
         ('Test 11 Check join class, missing parameter', test11, 'db/with_classes.db'),
         ('Test 12 Check leave class, missing parameter', test12, 'db/with_classes.db'),
         ('Test 13 Everyone should pass.', test13, 'db/with_classes.db'),
         ('Test 14 Get class, normal.', test14, 'db/with_classes.db'),
         ('Test 15 Get class, no attendees.', test15, 'db/with_classes.db'),
         ('Test 16 Get class, not trainer.', test16, 'db/with_classes.db'),
         ('Test 17 Get class, no such class.', test17, 'db/with_classes.db'),
         ('Test 18 Set update pass.', test18, 'db/with_class3_update.db'),
         ('Test 19 Set update not trainer', test19, 'db/with_class3_update.db'),
         ('Test 20 Set update too soon', test20, 'db/with_class3_update.db'),
         ('Test 21 cancel class successful.', test21, 'db/with_classes.db'),
         ('Test 22 cancel class not trainer.', test22, 'db/with_classes.db'),
         ('Test 23 cancel class to late.', test23, 'db/with_classes.db'),
         ('Test 24 cancel class missing id.', test24, 'db/with_classes.db'),
         ('Test 25 cancel class invalid.', test25, 'db/with_classes.db'),
         ('Test 26 create class, success.', test26, 'db/with_classes.db'),
         ('Test 27 create class, not trainer', test27, 'db/with_classes.db'),
         ('Test 28 create class, invalid day.', test28, 'db/with_classes.db'),
         ('Test 29 create class, missing note', test29, 'db/with_classes.db'),
         ('Test 30 create class, not a skill.', test30, 'db/with_classes.db'),
         ('Test 31 get skills.', test31, 'db/skill_records.db'),
         ('Test 32 get skills.', test32, 'db/skill_records.db'),
         ('Test 33 get skills.', test33, 'db/skill_records.db'),
         ('Test 34 get skills.', test34, 'db/skill_records.db'),
         ('Test 35 get skills.', test35, 'db/skill_records.db')
]
def test_suite(file_to_run):
    """Iterate over all the tests and run them with a fresh server and copy of the specified database"""
    global server_port # we cycle through ports if a program fails and occupies the port

    mark_total = 0 # how many tests pass
     
    # loop over the tests
    for tnumber, test in enumerate(tests, start = 1):
        
        #make sure we are using a free port
        port = int(server_port)
        busy = 1
        while busy == 1:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(2)
            result = sock.connect_ex(('localhost',port))
            if result == 0:
                print('Port',port,'busy')
                port +=1
            else:
                print('Using port',port)
                busy = 0
                
        server_port = str(port)
        
        print("Running",test[0])
        
        # try the test, if it generates an exception that is considered a fail.
        try:
            # copy the database to where the server expects it
            shutil.copy(test[2],'database.db') 
            
            #start the server
            sp = subprocess.Popen('C:/Users/jong0/AppData/Local/Microsoft/WindowsApps/PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0/python.exe '+ file_to_run +' '+server_port)
            
            # run the test and record the mark.
            tmark = test[1]()
            mark_total += tmark

            # close down the server
            print("Test Finished")
            sp.terminate()
            
        except Exception as err:
            print(err)
            try:
                sp.terminate()
            except:
                pass
            print("Test Process Generated Exception")
        tnumber += 1
        
    print("Marks = {:d}/35 for regression tests.".format(mark_total))

server_port=8080
file_to_run = 'server.py'
test_suite(file_to_run)
# use pylint to check the mark, a problem is indicated by -1
print("Pylint")
print(check_pylint(file_to_run))

Using port 8080
Running Test 1 Simple login test.
Test 1 Passed
Test Finished
Port 8080 busy
Using port 8081
Running Test 2 Check bad password rejected.
Test 2 Passed
Test Finished
Port 8081 busy
Using port 8082
Running Test 3 Check missing username on login is rejected
Test 3 Passed
Test Finished
Port 8082 busy
Using port 8083
Running Test 4 Check magic is different
Test 4 Passed
Test Finished
Port 8083 busy
Using port 8084
Running Test 5 Check upcoming when future classes
Test 5 Failed - Expected number of class responses not found.
Test Finished
Port 8084 busy
Using port 8085
Running Test 6 Check when no classes
Test 6 Passed
Test Finished
Port 8085 busy
Using port 8086
Running Test 7 Check when no future classes
Test 7 Passed
Test Finished
Port 8086 busy
Using port 8087
Running Test 8 Check join class
Test 8 Passed
Test Finished
Port 8087 busy
Using port 8088
Running Test 9 Check join non-existant class
Test 9 Passed
Test Finished
Port 8088 busy
Using port 8089
Running Test 10 Chec